In [7]:
import pandas as pd
from datetime import datetime
import os

# 1 Drop unimportant columns

def function_1(year):
    
    chunksize = 1000000
    i = 0
    header = True
    if (os.path.isfile(year + '_1_columndelete.csv')):
        os.remove(year + '_1_columndelete.csv')


    for chunk in pd.read_csv(year + '_Yellow_Taxi_Trip_Data.csv', chunksize = chunksize):
        i = i+1
        print(i)
        
        # For years with areas
        chunk[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']].to_csv(year + '_1_columndelete.csv', mode = 'a', header = header, index = False)

        # With tip_amount and total
        #chunk[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID','tip_amount','total_amount']].to_csv(year + '_1_columndelete.csv', mode = 'a', header = header, index = False)

        # !!!ONLY FOR 2016!!!
        #chunk[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','tip_amount','total_amount','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].to_csv(year + '_1_columndelete.csv', mode = 'a', header = header, index = False)

        header = False

    print('finished 1')




# 2 check time-window (all-in-one)

def function_2(year):

    if (os.path.isfile(year + '_2_complete.csv')):
        os.remove(year + '_2_complete.csv')
    chunksize = 1000
    i = 0
    header = True

    compare_am = datetime.strptime('06:00:00 AM', '%I:%M:%S %p').time()
    compare_pm = datetime.strptime('06:00:00 PM', '%I:%M:%S %p').time()


    for chunk in pd.read_csv(year + '_1_columndelete.csv', chunksize = chunksize):
        counter = 0
        while (counter < chunksize):
            try:
                string_pickup = chunk['tpep_pickup_datetime'][i]
                help_time_start = datetime.strptime(string_pickup,'%m/%d/%Y %I:%M:%S %p').time()
                string_dropoff = chunk['tpep_dropoff_datetime'][i]
                help_time_end = datetime.strptime(string_dropoff,'%m/%d/%Y %I:%M:%S %p').time()

                if (compare_am > help_time_start or compare_pm <= help_time_start or compare_am > help_time_end or compare_pm <= help_time_end):
                    chunk = chunk.drop([i])
            except:
                pass
            i =  i + 1
            counter = counter + 1
        chunk.to_csv(year + '_2_complete.csv', mode = 'a', header = header, index = False)
        header = False
        print(i)

    print('finished 2 complety')

                 
                     
                     
# !!!NOT NECESSARY!!! 3 Check if startzone == endzone, then delete row
                     
def function_3(year):
    if (os.path.isfile(year + '_3_deletesamezone.csv')):
        os.remove(year + '_3_deletesamezone.csv')
    chunksize = 1000
    i = 0
    header = True
    rowsdropped = 0
    compare_am = datetime.strptime('06:00:00', '%H:%M:%S').time()

    for chunk in pd.read_csv(year + '_2_complete.csv', chunksize = chunksize):
        counter = 0
        print(i)
        while (counter < chunksize):            
            try:
                startzone = chunk['PULocationID'][i]
                endzone = chunk['DOLocationID'][i]

                # Check if zones are equal
                if (startzone == endzone):        
                    chunk = chunk.drop([i])
            except:
                pass

            i += 1
            counter +=  1
        chunk.to_csv(year + '_3_deletesamezone.csv', mode = 'a', header = header, index = False)
        header = False
    print('finished 3')


# !!!NOT NECESSARY !!!  4 Costs = Total costs - Tip, delete tip + total amount

def function_4(year):
    if (os.path.isfile(year + '_4_calculate_costs.csv')):
        os.remove(year + '_4_calculate_costs.csv')
    chunksize = 1000
    i = 0
    header = True

    for chunk in pd.read_csv(year + '_3_deletesamezone.csv', chunksize = chunksize):
        counter = 0
        print(i)
        while (counter < chunksize):
            try:
                chunk['costs'] = chunk['total_amount'] - chunk['tip_amount']
            except:
                pass
            i += 1
            counter +=  1
        
        # Delete total_amount and tip_amount
        df[list("ABCD")].astype(int)
        chunk = chunk.drop(columns=['total_amount', 'tip_amount'])
        
        chunk.to_csv(year + '_4_calculate_costs.csv', mode = 'a', header = header, index = False)
        header = False
    print('finished 4')

    
    
    
# 5 duration = timeend - timestart
    
def function_5(year):    
    
    if (os.path.isfile(year + '_5_add_duration.csv')):
        os.remove(year + '_5_add_duration.csv')
    
    header = True
    chunksize = 1000
    i = 0

    for chunk in pd.read_csv(year + '_2_complete.csv', chunksize = chunksize):

        #Get duration of trip
        counter = 0
        print(i)
        chunk['time'] = -1
        while (counter < chunksize):
            try:
                string_pickup = chunk['tpep_pickup_datetime'][i]
                datetime_pickup = datetime.strptime(string_pickup,'%m/%d/%Y %I:%M:%S %p')
                string_dropoff = chunk['tpep_dropoff_datetime'][i]
                datetime_dropoff = datetime.strptime(string_dropoff,'%m/%d/%Y %I:%M:%S %p')
                # add time to data as new column
                chunk['time'][i] = datetime_dropoff - datetime_pickup
            
            except:
                pass
            
            i =  i + 1
            counter = counter + 1
            
        chunk.to_csv(year + '_5_add_duration.csv', mode = 'a', header = header, index = False)
        header = False
    print('finished 5')
    
#cleaning trips longer than 5h
    
def function_6(year):
    if (os.path.isfile(year + '_6_morethan5hours.csv')):
        os.remove(year + '_6_morethan5hours.csv')
    chunksize = 1000
    i = 0
    header = True
    rowsdropped = 0
    for chunk in pd.read_csv(year + '_5_add_duration.csv', chunksize = chunksize):
        counter = 0
        print(i)
        while (counter < chunksize):            
            try:
                time = chunk['time'][i]
                t = datetime.strptime(time,"%H:%M:%S")
                delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
                compare = timedelta(hours=5, minutes=0, seconds=0)
                if (delta > compare):                
                    chunk = chunk.drop([i])               
            except:
                chunk = chunk.drop([i])

            i += 1
            counter +=  1
        chunk.to_csv(year + '_6_morethan5hours.csv', mode = 'a', header = header, index = False)
        header = False
    print('finished 6')
    

In [ ]:
print(datetime.now())            
#function_1('2019')
print(datetime.now())
#function_2('2019')
print(datetime.now())
#function_5('2019')
print(datetime.now())
function_6('2019')
print(datetime.now())

2021-04-29 09:21:10.599987
2021-04-29 09:21:10.599987
2021-04-29 09:21:10.599987
2021-04-29 09:21:10.599987
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
1430

1150000
1151000
1152000
1153000
1154000
1155000
1156000
1157000
1158000
1159000
1160000
1161000
1162000
1163000
1164000
1165000
1166000
1167000
1168000
1169000
1170000
1171000
1172000
1173000
1174000
1175000
1176000
1177000
1178000
1179000
1180000
1181000
1182000
1183000
1184000
1185000
1186000
1187000
1188000
1189000
1190000
1191000
1192000
1193000
1194000
1195000
1196000
1197000
1198000
1199000
1200000
1201000
1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
1229000
1230000
1231000
1232000
1233000
1234000
1235000
1236000
1237000
1238000
1239000
1240000
1241000
1242000
1243000
1244000
1245000
1246000
1247000
1248000
1249000
1250000
1251000
1252000
1253000
1254000
1255000
1256000
1257000
1258000
1259000
1260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000
1270000
1271000
1272000
1273000
1274000


2175000
2176000
2177000
2178000
2179000
2180000
2181000
2182000
2183000
2184000
2185000
2186000
2187000
2188000
2189000
2190000
2191000
2192000
2193000
2194000
2195000
2196000
2197000
2198000
2199000
2200000
2201000
2202000
2203000
2204000
2205000
2206000
2207000
2208000
2209000
2210000
2211000
2212000
2213000
2214000
2215000
2216000
2217000
2218000
2219000
2220000
2221000
2222000
2223000
2224000
2225000
2226000
2227000
2228000
2229000
2230000
2231000
2232000
2233000
2234000
2235000
2236000
2237000
2238000
2239000
2240000
2241000
2242000
2243000
2244000
2245000
2246000
2247000
2248000
2249000
2250000
2251000
2252000
2253000
2254000
2255000
2256000
2257000
2258000
2259000
2260000
2261000
2262000
2263000
2264000
2265000
2266000
2267000
2268000
2269000
2270000
2271000
2272000
2273000
2274000
2275000
2276000
2277000
2278000
2279000
2280000
2281000
2282000
2283000
2284000
2285000
2286000
2287000
2288000
2289000
2290000
2291000
2292000
2293000
2294000
2295000
2296000
2297000
2298000
2299000


3200000
3201000
3202000
3203000
3204000
3205000
3206000
3207000
3208000
3209000
3210000
3211000
3212000
3213000
3214000
3215000
3216000
3217000
3218000
3219000
3220000
3221000
3222000
3223000
3224000
3225000
3226000
3227000
3228000
3229000
3230000
3231000
3232000
3233000
3234000
3235000
3236000
3237000
3238000
3239000
3240000
3241000
3242000
3243000
3244000
3245000
3246000
3247000
3248000
3249000
3250000
3251000
3252000
3253000
3254000
3255000
3256000
3257000
3258000
3259000
3260000
3261000
3262000
3263000
3264000
3265000
3266000
3267000
3268000
3269000
3270000
3271000
3272000
3273000
3274000
3275000
3276000
3277000
3278000
3279000
3280000
3281000
3282000
3283000
3284000
3285000
3286000
3287000
3288000
3289000
3290000
3291000
3292000
3293000
3294000
3295000
3296000
3297000
3298000
3299000
3300000
3301000
3302000
3303000
3304000
3305000
3306000
3307000
3308000
3309000
3310000
3311000
3312000
3313000
3314000
3315000
3316000
3317000
3318000
3319000
3320000
3321000
3322000
3323000
3324000


4225000
4226000
4227000
4228000
4229000
4230000
4231000
4232000
4233000
4234000
4235000
4236000
4237000
4238000
4239000
4240000
4241000
4242000
4243000
4244000
4245000
4246000
4247000
4248000
4249000
4250000
4251000
4252000
4253000
4254000
4255000
4256000
4257000
4258000
4259000
4260000
4261000
4262000
4263000
4264000
4265000
4266000
4267000
4268000
4269000
4270000
4271000
4272000
4273000
4274000
4275000
4276000
4277000
4278000
4279000
4280000
4281000
4282000
4283000
4284000
4285000
4286000
4287000
4288000
4289000
4290000
4291000
4292000
4293000
4294000
4295000
4296000
4297000
4298000
4299000
4300000
4301000
4302000
4303000
4304000
4305000
4306000
4307000
4308000
4309000
4310000
4311000
4312000
4313000
4314000
4315000
4316000
4317000
4318000
4319000
4320000
4321000
4322000
4323000
4324000
4325000
4326000
4327000
4328000
4329000
4330000
4331000
4332000
4333000
4334000
4335000
4336000
4337000
4338000
4339000
4340000
4341000
4342000
4343000
4344000
4345000
4346000
4347000
4348000
4349000


...

7300000
7301000
7302000
7303000
7304000
7305000
7306000
7307000
7308000
7309000
7310000
7311000
7312000
7313000
7314000
7315000
7316000
7317000
7318000
7319000
7320000
7321000
7322000
7323000
7324000
7325000
7326000
7327000
7328000
7329000
7330000
7331000
7332000
7333000
7334000
7335000
7336000
7337000
7338000
7339000
7340000
7341000
7342000
7343000
7344000
7345000
7346000
7347000
7348000
7349000
7350000
7351000
7352000
7353000
7354000
7355000
7356000
7357000
7358000
7359000
7360000
7361000
7362000
7363000
7364000
7365000
7366000
7367000
7368000
7369000
7370000
7371000
7372000
7373000
7374000
7375000
7376000
7377000
7378000
7379000
7380000
7381000
7382000
7383000
7384000
7385000
7386000
7387000
7388000
7389000
7390000
7391000
7392000
7393000
7394000
7395000
7396000
7397000
7398000
7399000
7400000
7401000
7402000
7403000
7404000
7405000
7406000
7407000
7408000
7409000
7410000
7411000
7412000
7413000
7414000
7415000
7416000
7417000
7418000
7419000
7420000
7421000
7422000
7423000
7424000


8325000
8326000
8327000
8328000
8329000
8330000
8331000
8332000
8333000
8334000
8335000
8336000
8337000
8338000
8339000
8340000
8341000
8342000
8343000
8344000
8345000
8346000
8347000
8348000
8349000
8350000
8351000
8352000
8353000
8354000
8355000
8356000
8357000
8358000
8359000
8360000
8361000
8362000
8363000
8364000
8365000
8366000
8367000
8368000
8369000
8370000
8371000
8372000
8373000
8374000
8375000
8376000
8377000
8378000
8379000
8380000
8381000
8382000
8383000
8384000
8385000
8386000
8387000
8388000
8389000
8390000
8391000
8392000
8393000
8394000
8395000
8396000
8397000
8398000
8399000
8400000
8401000
8402000
8403000
8404000
8405000
8406000
8407000
8408000
8409000
8410000
8411000
8412000
8413000
8414000
8415000
8416000
8417000
8418000
8419000
8420000
8421000
8422000
8423000
8424000
8425000
8426000
8427000
8428000
8429000
8430000
8431000
8432000
8433000
8434000
8435000
8436000
8437000
8438000
8439000
8440000
8441000
8442000
8443000
8444000
8445000
8446000
8447000
8448000
8449000


9350000
9351000
9352000
9353000
9354000
9355000
9356000
9357000
9358000
9359000
9360000
9361000
9362000
9363000
9364000
9365000
9366000
9367000
9368000
9369000
9370000
9371000
9372000
9373000
9374000
9375000
9376000
9377000
9378000
9379000
9380000
9381000
9382000
9383000
9384000
9385000
9386000
9387000
9388000
9389000
9390000
9391000
9392000
9393000
9394000
9395000
9396000
9397000
9398000
9399000
9400000
9401000
9402000
9403000
9404000
9405000
9406000
9407000
9408000
9409000
9410000
9411000
9412000
9413000
9414000
9415000
9416000
9417000
9418000
9419000
9420000
9421000
9422000
9423000
9424000
9425000
9426000
9427000
9428000
9429000
9430000
9431000
9432000
9433000
9434000
9435000
9436000
9437000
9438000
9439000
9440000
9441000
9442000
9443000
9444000
9445000
9446000
9447000
9448000
9449000
9450000
9451000
9452000
9453000
9454000
9455000
9456000
9457000
9458000
9459000
9460000
9461000
9462000
9463000
9464000
9465000
9466000
9467000
9468000
9469000
9470000
9471000
9472000
9473000
9474000


10333000
10334000
10335000
10336000
10337000
10338000
10339000
10340000
10341000
10342000
10343000
10344000
10345000
10346000
10347000
10348000
10349000
10350000
10351000
10352000
10353000
10354000
10355000
10356000
10357000
10358000
10359000
10360000
10361000
10362000
10363000
10364000
10365000
10366000
10367000
10368000
10369000
10370000
10371000
10372000
10373000
10374000
10375000
10376000
10377000
10378000
10379000
10380000
10381000
10382000
10383000
10384000
10385000
10386000
10387000
10388000
10389000
10390000
10391000
10392000
10393000
10394000
10395000
10396000
10397000
10398000
10399000
10400000
10401000
10402000
10403000
10404000
10405000
10406000
10407000
10408000
10409000
10410000
10411000
10412000
10413000
10414000
10415000
10416000
10417000
10418000
10419000
10420000
10421000
10422000
10423000
10424000
10425000
10426000
10427000
10428000
10429000
10430000
10431000
10432000
10433000
10434000
10435000
10436000
10437000
10438000
10439000
10440000
10441000
10442000
10443000
1

11244000
11245000
11246000
11247000
11248000
11249000
11250000
11251000
11252000
11253000
11254000
11255000
11256000
11257000
11258000
11259000
11260000
11261000
11262000
11263000
11264000
11265000
11266000
11267000
11268000
11269000
11270000
11271000
11272000
11273000
11274000
11275000
11276000
11277000
11278000
11279000
11280000
11281000
11282000
11283000
11284000
11285000
11286000
11287000
11288000
11289000
11290000
11291000
11292000
11293000
11294000
11295000
11296000
11297000
11298000
11299000
11300000
11301000
11302000
11303000
11304000
11305000
11306000
11307000
11308000
11309000
11310000
11311000
11312000
11313000
11314000
11315000
11316000
11317000
11318000
11319000
11320000
11321000
11322000
11323000
11324000
11325000
11326000
11327000
11328000
11329000
11330000
11331000
11332000
11333000
11334000
11335000
11336000
11337000
11338000
11339000
11340000
11341000
11342000
11343000
11344000
11345000
11346000
11347000
11348000
11349000
11350000
11351000
11352000
11353000
11354000
1

In [ ]:
# Check outliers costs
import pandas as pd
from datetime import datetime
chunksize = 100000
num_out = 0
a = 0
# 2018_Yellow_Taxi_Trip_Data.csv,  2018_1_columndelete.csv, 2018_2_complete.csv, 2018_3_deletesamezone.csv, 2018_4_calculate_costs.csv, 2018_5_add_duration.csv

for chunk in pd.read_csv('2018_5_add_duration.csv', chunksize = chunksize):
    counter = 0
    while (counter < chunksize):
        #if (chunk['costs'][a] > 100 and chunk['trip_distance'][a] < 10):
        if (chunk['costs'][a] > 500):
            print(chunk.iloc[counter])
            num_out += 1
        a += 1
        counter += 1
        
    print(a, num_out)
    
print('There are ',counter, ' trips that are more expensive than 100 Dollar.')

In [ ]:
# Check outliers distance
import pandas as pd
from datetime import datetime
chunksize = 100000
num_out = 0
a = 0
# 2018_Yellow_Taxi_Trip_Data.csv,  2018_1_columndelete.csv, 2018_2_complete.csv, 2018_3_deletesamezone.csv, 2018_4_calculate_costs.csv, 2018_5_add_duration.csv

for chunk in pd.read_csv('2018_5_add_duration.csv', chunksize = chunksize):
    counter = 0
    while (counter < chunksize):
        if (chunk['trip_distance'][a] > 50):
            print(chunk.iloc[counter])
            num_out += 1
        a += 1
        counter += 1
        
    print(a, num_out)
    
print('There are ',counter, ' trips that are longer than 100km.')
            
    

In [ ]:
# Check outliers time

import pandas as pd
from datetime import datetime, timedelta
chunksize = 100000
num_out = 0
a = 0
# 2018_Yellow_Taxi_Trip_Data.csv,  2018_1_columndelete.csv, 2018_2_complete.csv, 2018_3_deletesamezone.csv, 2018_4_calculate_costs.csv, 2018_5_add_duration.csv

for chunk in pd.read_csv('2019_5_add_duration.csv', chunksize = chunksize):
    counter = 0
    while (counter < chunksize):
        try:
            time = chunk['time'][a]
            t = datetime.strptime(time,"%H:%M:%S")
            delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
            compare = timedelta(hours=5, minutes=0, seconds=0)
            if (delta > compare):
                #print(chunk.iloc[counter])
                num_out += 1
        except:
            pass
        a += 1
        counter += 1
        
    print(a, num_out)
    
print('There are ',counter, ' trips that are longer than 100km.')